In [2]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer, PreTrainedTokenizerFast

model = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-2.8B-deduped",
  revision="step143000",
  cache_dir="./pythia-2.8B-deduped/step143000",
).to("cuda")

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

c:\Users\TeamD\.conda\envs\gptj\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\Repos\Other\toolformer\pythia-6.9B-deduped\step143000. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/42.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 23.99 GiB total capacity; 22.89 GiB already allocated; 0 bytes free; 22.89 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
tokenizer: PreTrainedTokenizerFast = AutoTokenizer.from_pretrained(
  "EleutherAI/pythia-2.8B-deduped",
  revision="step143000",
  cache_dir="./pythia-2.8B-deduped/step143000",
)

In [3]:
num_added_toks = tokenizer.add_tokens(["[","]","->"], special_tokens=True)
print(num_added_toks)

0


In [4]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50277, 2560)

In [5]:
print(tokenizer.all_special_tokens) 
# doesn't show our new ones for some reason /shrug

['<|endoftext|>']


In [6]:
tokens = tokenizer("it is [NOW() -> 123]", return_tensors="pt").to("cuda")["input_ids"][0]
print(tokens)
print(tokenizer.convert_ids_to_tokens(tokens))

tensor([  262,   310,   209,    60, 22731,  1082,   209,  1168, 15567,    62],
       device='cuda:0')
['it', 'Ġis', 'Ġ', '[', 'NOW', '()', 'Ġ', '->', 'Ġ123', ']']


In [7]:
def list_windows():
    import psutil, win32process, win32gui, time
    def get_process_name(handle):
        pid = win32process.GetWindowThreadProcessId(handle) #This produces a list of PIDs active window relates to
        return (psutil.Process(pid[-1]).name()) #pid[-1] is the most likely to survive last longer

    rtn = []
    def winEnumHandler( hwnd, ctx ):
        nonlocal rtn
        if win32gui.IsWindowVisible( hwnd ):
            rtn.append((hwnd, win32gui.GetWindowText( hwnd ), get_process_name(hwnd)))
    win32gui.EnumWindows( winEnumHandler, None )
    return rtn
list_windows()

[(65924, '', 'explorer.exe'),
 (66074, '', 'explorer.exe'),
 (66052, '', 'explorer.exe'),
 (2689676, '● main.ipynb - toolformer - Visual Studio Code', 'Code.exe'),
 (327782, 'Task Manager', 'Taskmgr.exe'),
 (395466,
  '(14) Beethoven - Moonlight Sonata (FULL) - YouTube and 2 more pages - Personal - Microsoft\u200b Edge Dev',
  'msedge.exe'),
 (396024, 'Ideas - Visual Studio Code', 'Code.exe'),
 (132436, '@Chase - Discord', 'Discord.exe'),
 (133276, '', 'explorer.exe'),
 (67736, 'Clock', 'Time.exe'),
 (132478, 'Clock', 'ApplicationFrameHost.exe'),
 (132608, 'Groove Music', 'Music.UI.exe'),
 (328706, 'Groove Music', 'ApplicationFrameHost.exe'),
 (787954, 'Settings', 'SystemSettings.exe'),
 (198766, 'Settings', 'ApplicationFrameHost.exe'),
 (66224, '', 'explorer.exe'),
 (66216, '', 'explorer.exe'),
 (66198, '', 'explorer.exe'),
 (66180, '', 'explorer.exe'),
 (66150, '', 'explorer.exe'),
 (66148, '', 'explorer.exe'),
 (262364, 'Microsoft Text Input Application', 'TextInputHost.exe'),
 (661

In [8]:
def get_best_match(name):
    from fuzzywuzzy import fuzz
    x = sorted([(fuzz.ratio(str(x), name),x) for x in list_windows()], key=lambda x: x[0], reverse=True)
    print(x[0][1])
    return x[0][1][0]
get_best_match("VSCode")

(396024, 'Ideas - Visual Studio Code', 'Code.exe')


c:\Users\TeamD\.conda\envs\gptj\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


396024

In [9]:
def pos2pos(pos: str) -> (int,int,int,int):
    from screeninfo import get_monitors
    monitors = get_monitors()
    left = sorted(monitors, key=lambda x: x.x)[0]
    right = sorted(monitors, key=lambda x: x.x, reverse=True)[0]
    main = [x for x in monitors if x.is_primary][0]
    corners = {
        "top left": lambda m: (m.x, m.y, m.width//2, m.height//2),
        "top right": lambda m: (m.x + m.width//2, m.y, m.width//2, m.height//2),
        "bottom left": lambda m: (m.x, m.y + m.height//2, m.width//2, m.height//2),
        "bottom right": lambda m: (m.x + m.width//2, m.y + m.height//2, m.width//2, m.height//2),
    }
    lookup = {}
    for corner, func in corners.items():
        lookup[f"left monitor, {corner}"] = func(left)
        lookup[f"third monitor, {corner}"] = func(left)
        
        lookup[f"main monitor, {corner}"] = func(main)

        lookup[f"right monitor, {corner}"] = func(right)
        lookup[f"second monitor, {corner}"] = func(right)
        lookup[f"secondary monitor, {corner}"] = func(right)
    
    lookup[f"main monitor, full"] = (main.x, main.y, main.width, main.height)
    
    lookup[f"left monitor, full"] = (left.x, left.y, left.width, left.height)
    lookup[f"third monitor, full"] = (left.x, left.y, left.width, left.height)
    
    lookup[f"right monitor, full"] = (right.x, right.y, right.width, right.height)
    lookup[f"secondary monitor, full"] = (right.x, right.y, right.width, right.height)

    from fuzzywuzzy import fuzz
    x = sorted([(fuzz.ratio(label, pos),data) for label,data in lookup.items()], key=lambda x: x[0], reverse=True)
    # print(x)
    return x[0][1]
    
print(pos2pos("right monitor, bottom left"))
print(pos2pos("main monitor, full"))

(1920, 540, 960, 540)
(0, 0, 1920, 1080)


In [10]:
def snap(params):
    window, position = params.split(",")

    import ctypes
    user32 = ctypes.windll.user32

    user32.SetProcessDPIAware()
    res = (user32.GetSystemMetrics(0), user32.GetSystemMetrics(1))
    print("found resolution: ", res)
    handle = get_best_match(window)
    # handle = user32.FindWindowW(None, u'Untitled - Notepad')
    if handle == 0:
        return "could not find window"
    # move window using handle
    # MoveWindow(handle, x, y, height, width, repaint(bool))
    pos = pos2pos(position)
    user32.MoveWindow(handle, *pos, True)
    return f"{window} to {pos}"

In [11]:
snap("notepad, second monitor bottom left")

found resolution:  (1920, 1080)
(328706, 'Groove Music', 'ApplicationFrameHost.exe')


'notepad to (1920, 540, 960, 540)'

In [12]:
known_functions = {
    "SNAP": lambda x: snap(x),
}

In [13]:
def invoke(response):
    import re
    pattern = r".*\[(\w+)\((.*?)\s*\)\s*->"
    print(re.findall(pattern, response, re.DOTALL))
    x = re.findall(pattern, response)
    if len(x) == 0:
        return "bad match"
    func, params = x[0]

    if func not in known_functions:
        return f"Unknown function {func}"
    print(f"Calling {func} with {params}")
    result = known_functions[func](params)
    return result

In [29]:
def predict(prompt: str, eos: str) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        # do_sample=True,
        temperature=0.9,
        max_new_tokens=100,
        eos_token_id=tokenizer.encode(eos)[0],
    )
    response = tokenizer.decode(outputs[:,inputs["input_ids"].shape[1]:][0])
    return response

In [27]:
response = predict("19+10=", eos="]")
print(response)

Setting `pad_token_id` to `eos_token_id`:62 for open-end generation.


0.

A:

The problem is that you are using the wrong definition of the sum of a series.
The sum of a series is defined as the limit of the partial sums.
So, if you have a series $a_n$ with $a_n = \frac{1}{n}$ for $n \in \mathbb{N}$, then the sum of this series is $\sum_{n=1}^\infty \frac{1}{n} =
tensor([[  746,    12,   740,    30,    17,    15,   187,   187,    34,    27,
           187,   187,   510,  1895,   310,   326,   368,   403,   970,   253,
          3430,  5426,   273,   253,  2020,   273,   247,  2962,    15,   187,
           510,  2020,   273,   247,  2962,   310,  2931,   347,   253,  2701,
           273,   253,  7898, 22661,    15,   187,  2598,    13,   604,   368,
           452,   247,  2962,   370,    66,    64,    79,     5,   342,   370,
            66,    64,    79,   426,   393,  1124,    92,    18,  1217,    79,
           724,   323,   370,    79,   393,   249,   393,  1991,    92,    47,
          3303,   840,   253,  2020,   273,   436,  2962,   310,   669, 

In [1]:
from tools.helpers import handle_tool_invocation, build_preprompt
print(build_preprompt())

You are an AI assistant with several tools available to you. The tools are the following:
NOW: Print the current time.

DO NOT USE TOOLS WITHIN TOOLS! KEEP ALL TOOL CALLS SEPARATE FROM EACH OTHER!

User: What time is it?
Assistant: It is [NOW() -> 2021-07-23 11:51:08.123456] 11:51 AM.
User: What is the date?
Assistant: It is [NOW() -> 2021-07-23 11:51:08.123456] 2021-07-23.
User: 


In [54]:
pre_prompt = r"""You are an AI assistant with several tools available to you. The tools are the following:
MATH: This allows you to evaluate mathematical expressions using the math.js library.
NOW: This returns the current date and time. You must not pass any arguments to this tool!
SEARCH: This returns a web search result for the given string argument.
REMEMBER: This stores a key-value pair in the assistant's memory. You must pass two arguments to this tool, the first being the key, and the second being the value.
RECALL: This returns a value from the assistant's memory. You must pass one argument to this tool, the key.
OPEN: This opens a file or a website. The only argument is the URI for the file or website.
SNAP: This moves a window to an area of the screen. The first argument is the program, and the second argument is the position.

DO NOT USE TOOLS WITHIN TOOLS! KEEP ALL TOOL CALLS SEPARATE FROM EACH OTHER!

User: What is 10 times 14?
Assistant:10 * 14 is [MATH(10 * 14) -> 140] 140.
User: Has Avatar the way of water been released yet?
Assistant:[SEARCH(Avatar the way of water release date) -> 22.11.2022] Avatar: The way of water was released on the 22nd of november 2022. Today is [NOW() -> Mon Feb 27 2023] the 13th of February 2023. Therefore, [MATH(2023 > 2022) -> true] it was released last year.
User: What day is it?
Assistant:It is [NOW() -> 13/02/2023, 20:01:35] the 13th of February 2023.
User: How old is Donald Trump?
Assistant:Donald Trump is [SEARCH(Donald Trump age) -> 76] 76 years old.
User: Where was the inventor of the light bulb born, and what is the last country his home country was at war with?
Assistant:Thomas Edison, the inventor of the lightbulb, was born in [SEARCH(Thomas Edison birthplace) -> Milan, Ohio] Milan, Ohio. The last country the United States was at war with was [SEARCH(last country US at war with) -> Iraq] Iraq.
User: Set my default browser to Firefox.
Assistant:Okay, I set your default browser to [REMEMBER(default browser, Firefox) -> Firefox].
User: Open youtube.
Assistant: Okay, opening YouTube... [OPEN(https://youtube.com) -> 0] done!
User: Open my csgo config.
Assistant: Okay, opening your csgo config... [OPEN(C:\Program Files (x86)\Steam\steamapps\common\Counter-Strike Global Offensive\csgo\cfg\autoexec.cfg) -> 0] done!
User: Move chrome to my left monitor.
Assistant: Okay, moving chrome... [SNAP(Chrome, left monitor) -> 0] done!
User: Move spotify to the top right corner of my second monitor.
Assistant: Okay, moving spotify... [SNAP(Spotify, right monitor top right) -> 0] done!
User: Move edge to the main monitor, top right.
Assistant: Okay, moving edge... [SNAP(Edge, main monitor top right) -> 0] done!
User: Move discord to my third monitor
Assistant: Okay, moving discord... [SNAP(Discord, third monitor) -> 0] done!
User: move terminal, secondary monitor, top right
Assistant: Okay, moving terminal... [SNAP(Terminal, second monitor top right) -> 0] done!
User: move notepad, main monitor
Assistant: Okay, moving notepad... [SNAP(Notepad, main monitor full) -> 0] done!
User: """

# prompt = pre_prompt + "move notepad, main monitor"
prompt = pre_prompt + input() + "\n"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
# print(prompt)
tokens = model.generate(
    **inputs,
    # do_sample=True,
    temperature=0.9,
    max_new_tokens=500,
    eos_token_id=tokenizer.encode("->")[0],
)
response = tokenizer.decode(tokens[:,inputs["input_ids"].shape[1]:][0])
output = invoke(response)
prompt = prompt + response + output + "]"
# print(prompt)
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
print(prompt)
tokens = model.generate(
    **inputs,
    # do_sample=True,
    temperature=0.9,
    max_new_tokens=100,
    eos_token_id=tokenizer.encode("\n")[0],
)
response = tokenizer.decode(tokens[:,inputs["input_ids"].shape[1]:][0])
print(response)


Setting `pad_token_id` to `eos_token_id`:1168 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:187 for open-end generation.


[('SNAP', 'Notepad, right monitor bottom right')]
Calling SNAP with Notepad, right monitor bottom right
found resolution:  (1920, 1080)
(1183534, 'Untitled - Notepad', 'notepad.exe')
You are an AI assistant with several tools available to you. The tools are the following:
MATH: This allows you to evaluate mathematical expressions using the math.js library.
NOW: This returns the current date and time. You must not pass any arguments to this tool!
SEARCH: This returns a web search result for the given string argument.
REMEMBER: This stores a key-value pair in the assistant's memory. You must pass two arguments to this tool, the first being the key, and the second being the value.
RECALL: This returns a value from the assistant's memory. You must pass one argument to this tool, the key.
OPEN: This opens a file or a website. The only argument is the URI for the file or website.
SNAP: This moves a window to an area of the screen. The first argument is the program, and the second argument is